In [65]:
import pandas as pd
import numpy as np
from math import sqrt
#https://www.kaggle.com/arashnic/book-recommendation-dataset/tasks

In [112]:
#create a book dataframe
books_df = pd.read_csv("./Books.csv")

#create a rating dataframe
ratings_df = pd.read_csv("./Ratings.csv")

D:\pyy\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [113]:
books_df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [114]:
#remove unimportant columns from books dataframe
books_df = books_df.drop(["Image-URL-S", "Image-URL-M", "Image-URL-L"], axis=1)

In [115]:
books_df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


In [116]:
#check for any null value and remove row if the number of null values is not significant
print(books_df.isnull().sum())
books_df.dropna(axis=0, how="any", inplace=True)

ISBN                   0
Book-Title             0
Book-Author            1
Year-Of-Publication    0
Publisher              2
dtype: int64


In [117]:
books_df.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            0
Year-Of-Publication    0
Publisher              0
dtype: int64

In [7]:
ratings_df.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [119]:
print(ratings_df.isnull().sum())

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64


In [21]:
ratings_df.shape

(1149780, 3)

In [34]:
#get top people who rate books frequently
topuser = ratings_df[['User-ID', 'Book-Rating']].groupby("User-ID").size().sort_values(ascending=False)
topuser.head(10)

User-ID
11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
212898     4785
278418     4533
76352      3367
110973     3100
235105     3067
dtype: int64

In [8]:
#create user dataframe
user_df = pd.read_csv("./Users.csv")
user_df.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [120]:
print(user_df.isnull().sum())

User-ID          0
Location         0
Age         110762
dtype: int64


In [121]:
user_df.shape

(278858, 3)

In [122]:
#pick one of the top 10 to recommend a book
input_user = ratings_df[ratings_df["User-ID"] ==110973]
input_user.drop(["User-ID"], axis=1, inplace=True)

#sampling that user rating's history
input_user.sample(n=10, random_state=41)

D:\pyy\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,ISBN,Book-Rating
465270,0886779146,9
463989,0425168964,7
465369,1568655800,9
464741,0671535358,0
462631,0345348656,8
462406,0070621993,7
465230,0843953276,0
463670,0373251017,0
465356,1560252774,0
465329,1551664976,0


In [123]:
#find users that are similar to the target
userSubset = ratings_df[ratings_df['ISBN'].isin(input_user['ISBN'].tolist())]
userSubset.head()

,User-ID,ISBN,Book-Rating
10,276746,0425115801,0
12,276746,0553561618,0
28,276754,0684867621,8
29,276755,0451166892,5
34,276762,0451167317,0


In [124]:
userSubsetGroup = userSubset.groupby(['User-ID'])

#pick a random group
userSubsetGroup.get_group(98391)

,User-ID,ISBN,Book-Rating
405994,98391,0060198702,9
406217,98391,0066214122,8
406428,98391,0312284683,9
406837,98391,0312872380,9
406838,98391,0312872399,8
406890,98391,0312877420,9
406917,98391,0312879016,8
407469,98391,0373710186,9
407525,98391,0373711344,9
407722,98391,0375504397,10


In [125]:
#Sorting it so users with movie most in common with the input will have priority
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)

In [126]:
#calculating personCorr
pearsonCorrelationDict = {}

#For every user group in our subset
for name, group in userSubsetGroup:
    #Let's start by sorting the input and current user group so the values aren't mixed up later on
    group = group.sort_values(by='ISBN')
    input_user = input_user.sort_values(by='ISBN')
    #Get the N for the formula
    nRatings = len(group)
    #Get the review scores for the movies that they both have in common
    temp_df = input_user[input_user['ISBN'].isin(group['ISBN'].tolist())]
    #And then store them in a temporary buffer variable in a list format to facilitate future calculations
    tempRatingList = temp_df['Book-Rating'].tolist()
    #Let's also put the current user group reviews in a list format
    tempGroupList = group['Book-Rating'].tolist()
    #Now let's calculate the pearson correlation between two users, so called, x and y
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
    
    #If the denominator is different than zero, then divide, else, 0 correlation.
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0

In [127]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['User-ID'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

,similarityIndex,User-ID
0,1.000000,110973
1,0.000000,203968
2,-0.005947,11676
3,0.080148,35859
4,0.000000,198711


In [128]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head()

,similarityIndex,User-ID
3663,1.0,92152
3768,1.0,115700
3550,1.0,62844
4204,1.0,213531
4041,1.0,172406


In [129]:
topUsersRating=topUsers.merge(ratings_df, left_on='User-ID', right_on='User-ID', how='inner')
topUsersRating.head()

,similarityIndex,User-ID,ISBN,Book-Rating
0,1.0,92152,0007154615,10
1,1.0,92152,0060168609,0
2,1.0,92152,0060935332,0
3,1.0,92152,0140075623,10
4,1.0,92152,014014773X,0


In [130]:
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['Book-Rating']
topUsersRating.head()

,similarityIndex,User-ID,ISBN,Book-Rating,weightedRating
0,1.0,92152,0007154615,10,10.0
1,1.0,92152,0060168609,0,0.0
2,1.0,92152,0060935332,0,0.0
3,1.0,92152,0140075623,10,10.0
4,1.0,92152,014014773X,0,0.0


In [131]:
tempTopUsersRating = topUsersRating.groupby('ISBN').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
ISBN,,
0 7336 1053 6,1.0,0.0
0002247127,1.0,0.0
0006130380,1.0,0.0
0006140378,1.0,7.0
0006170552,1.0,0.0


In [132]:
#Creates an empty dataframe
recommendation_df = pd.DataFrame()
#Now we take the weighted average
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['ISBN'] = tempTopUsersRating.index
recommendation_df.head()

,weighted average recommendation score,ISBN
ISBN,,
0 7336 1053 6,0.0,0 7336 1053 6
0002247127,0.0,0002247127
0006130380,0.0,0006130380
0006140378,7.0,0006140378
0006170552,0.0,0006170552


In [133]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(10)

,weighted average recommendation score,ISBN
ISBN,,
096265342X,10.0,096265342X
0553802496,10.0,0553802496
0452267218,10.0,0452267218
1569472602,10.0,1569472602
0345378881,10.0,0345378881
156830370X,10.0,156830370X
0345382846,10.0,0345382846
1567311199,10.0,1567311199
0395193958,10.0,0395193958


In [134]:
books_df.loc[books_df['ISBN'].isin(recommendation_df.head(10)['ISBN'].tolist())]

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
1763,0395177111,The Hobbit (Leatherette Collector's Edition),J. R. R. Tolkien,1973,Houghton Mifflin Company
3586,0345382846,Here Be Dragons,Sharon Kay Penman,1993,Ballantine Books
4385,1567311199,The Haunted Omnibus,Ed Laing,1997,MJF Books
6028,0553802496,Odd Thomas,Dean Koontz,2003,Bantam
15218,0395193958,The Lord of the Rings (Leatherette Collector's...,J. R. R. Tolkien,1974,Houghton Mifflin Company
16311,096265342X,Principia Discordia,Omar K. Ravenhurst,1991,Illuminet Press
70868,0345378881,The Reckoning,SHARON KAY PENMAN,1992,Ballantine Books
84851,0452267218,Salem's Lot (The Stephen King collectors edition),Stephen King,1991,Penguin USA
188094,1569472602,The Horus Road: Lords of the Two Lands: Volume 3,Pauline Gedge,2001,Soho Press
229923,156830370X,Adobe(R) PageMaker(R) 6.5 Classroom in a Book,Adobe Creative Team,1997,Adobe Press


In [106]:
#Quick check of target user high rated books
#Can see that the recommended books are quite similar to target user's favourite books (fictions, war stories, etc..)
input_user = input_user[input_user['Book-Rating'] == 10]
books_df.loc[books_df['ISBN'].isin(input_user['ISBN'].tolist())]

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
1207,0380789019,Neverwhere,Neil Gaiman,1998,Avon
1472,0671527215,Hitchhikers's Guide to the Galaxy,Douglas Adams,1984,Pocket
1507,0375703055,Ellen Foster,Kaye Gibbons,1997,Vintage Books
1518,0070212570,Fried Green Tomatoes at the Whistle Stop Cafe,Fannie Flagg,1989,McGraw-Hill
1763,0395177111,The Hobbit (Leatherette Collector's Edition),J. R. R. Tolkien,1973,Houghton Mifflin Company
2122,0345350499,The Mists of Avalon,MARION ZIMMER BRADLEY,1987,Del Rey
3329,0060502258,The Divine Secrets of the Ya-Ya Sisterhood: A ...,Rebecca Wells,2002,HarperTorch
3343,0451455126,Night Screams: Twenty-Two Stories of Terror,Edward Gorman,1996,Roc
3357,0385199570,The Stand (The Complete and Uncut Edition),Stephen King,1990,Doubleday Books
3586,0345382846,Here Be Dragons,Sharon Kay Penman,1993,Ballantine Books
